<a href="https://colab.research.google.com/github/XaviAlvarado18/Cuda-Project/blob/main/Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!nvidia-smi


Mon Nov 11 03:21:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [24]:
!apt-get update
!apt-get install -y build-essential
!apt-get install -y cuda


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [25]:
!apt-get update
!apt-get install -y libjpeg-dev


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [26]:
%%writefile pgm.cpp

/*/============================================================================
 Author        : G. Barlas
 Version       : 1.0
 Last modified : December 2014
 License       : Released under the GNU GPL 3.0
 Description   : Used in different projects to handle PGM I/O
 To build use  :
 ============================================================================
 */
#include <stdio.h>
#include <string.h>
#include <assert.h>
#include <jpeglib.h>
#include <math.h>
#include <vector>
#include "pgm.h"

using namespace std;

//-------------------------------------------------------------------
PGMImage::PGMImage(char *fname, int programID)
{
   if(programID == 0) {
        this->color = {255,0,0};
   } else if(programID == 1) {
        this->color = {0,255,0};
   } else {
        this->color = {0,0,255};
   }
   this->x_dim=this->y_dim=this->num_colors=0;
   this->pixels=NULL;

   FILE *ifile;
   ifile = fopen(fname, "rb");
   if(!ifile) return;

   char *buff = NULL;
   size_t temp;

   fscanf(ifile, "%*s %i %i %i", &this->x_dim, &this->y_dim, &this->num_colors);

   getline((char **)&buff, &temp, ifile); // eliminate CR-LF

   assert(this->x_dim >1 && this->y_dim >1 && this->num_colors >1);
   this->pixels = new unsigned char[this->x_dim * this->y_dim];
   fread((void *) this->pixels, 1, this->x_dim*this->y_dim, ifile);

   fclose(ifile);
}

// Constructor que acepta const char* y redirige al constructor existente
PGMImage::PGMImage(const char *filename) : PGMImage(const_cast<char *>(filename), 0) {
    // Este constructor redirige al existente que acepta char * y programID
}


//-------------------------------------------------------------------
// Definición del constructor en pgm.cpp sin valores predeterminados
PGMImage::PGMImage(int x, int y, int col) {
    this->num_colors = (col > 1) ? col : 16;
    this->x_dim = (x > 1) ? x : 100;
    this->y_dim = (y > 1) ? y : 100;
    this->pixels = new unsigned char[x_dim * y_dim];
    memset(this->pixels, 0, this->x_dim * this->y_dim);
    assert(this->pixels);
}
//-------------------------------------------------------------------
PGMImage::~PGMImage()
{
  if(this->pixels != NULL)
     delete [] this->pixels;
  this->pixels = NULL;
}

// Helper function to color a pixel in the given buffer
void colorPixel(unsigned char *colorPixels, int i, unsigned char r, unsigned char g, unsigned char b) {
    colorPixels[i*3] = r;
    colorPixels[i*3+1] = g;
    colorPixels[i*3+2] = b;
}

// Cambia el tipo de retorno de write a bool para que coincida con la declaración en pgm.h
bool PGMImage::write(const char* outputFileName, std::vector<std::pair<int, int>> selectedLines, float radIncrement, int rBins) {
    float maxRadius = hypot(this->x_dim, this->y_dim) / 2;
    float rScale = 2 * maxRadius / rBins;
    int xCenter = this->x_dim / 2;
    int yCenter = this->y_dim / 2;

    // Allocate memory for RGB image
    unsigned char *colorPixels = new unsigned char[this->x_dim * this->y_dim * 3];

    // Store pixel coordinates
    std::vector<std::pair<int, int>> coordinates(this->x_dim * this->y_dim);
    for (int i = 0; i < x_dim * y_dim; ++i) {
        coordinates[i] = std::make_pair(i % this->x_dim - xCenter, yCenter - i / this->x_dim);
    }

    for (int i = 0; i < this->x_dim * this->y_dim; ++i) {
        bool isLine = false;
        auto [x, y] = coordinates[i];

        for (const auto& line: selectedLines) {
            int rIdx = line.first;
            int thetaIdx = line.second;
            float r = rIdx * rScale - maxRadius;
            float theta = thetaIdx * radIncrement;

            if (std::abs(r - x * std::cos(theta) - y * std::sin(theta)) < 0.5) {
                isLine = true;
                break;
            }
        }
        // Verify if pixel needs to be coloured
        if (isLine) {
            colorPixel(colorPixels, i, this->color.at(0), this->color.at(1), this->color.at(2));  // Red color for line pixels
        } else {
            colorPixel(colorPixels, i, pixels[i], pixels[i], pixels[i]);  // Grayscale for non-line pixels
        }
    }

    // Configuración para compresión JPEG
    struct jpeg_compress_struct compressInfo;
    struct jpeg_error_mgr errorManager;

    // Open the output file
    FILE* outputFile = fopen(outputFileName, "wb");
    if (!outputFile) {
        printf("Error al abrir el archivo.\n");
        delete[] colorPixels;
        return false;
    }

    // Initialize the JPEG compression structures
    compressInfo.err = jpeg_std_error(&errorManager);
    jpeg_create_compress(&compressInfo);
    jpeg_stdio_dest(&compressInfo, outputFile);

    // Set the image properties
    compressInfo.image_width = this->x_dim;
    compressInfo.image_height = this->y_dim;
    compressInfo.input_components = 3; // RGB image
    compressInfo.in_color_space = JCS_RGB; // RGB color space

    // Set the JPEG compression parameters
    jpeg_set_defaults(&compressInfo);
    jpeg_set_quality(&compressInfo, 75, TRUE);

    // Start compression and write the image data
    jpeg_start_compress(&compressInfo, TRUE);
    JSAMPROW rowPointer[1];
    int rowStride = x_dim * 3;

    // Write each row
    while (compressInfo.next_scanline < compressInfo.image_height) {
        rowPointer[0] = &colorPixels[compressInfo.next_scanline * rowStride];
        jpeg_write_scanlines(&compressInfo, rowPointer, 1);
    }

    // Clean up after compression
    jpeg_finish_compress(&compressInfo);
    fclose(outputFile);
    jpeg_destroy_compress(&compressInfo);

    // Deallocate memory
    delete[] colorPixels;
    return true;
}

int PGMImage::getXDim(void) {
    return this->x_dim;
}

int PGMImage::getYDim(void) {
    return this->y_dim;
}

int PGMImage::getNumColors(void) {
    return this->num_colors;
}

unsigned char* PGMImage::getPixels(void) {
    return this->pixels;
}

Overwriting pgm.cpp


In [27]:
%%writefile pgm.h
#ifndef PGM_H
#define PGM_H

#include <vector>
#include <utility>

class PGMImage {
public:
    // Constructores
    PGMImage(const char *filename);           // Constructor para const char*
    PGMImage(char *fname, int programID);     // Constructor adicional con programID
    PGMImage(int x = 100, int y = 100, int col = 16); // Constructor con valores predeterminados
    ~PGMImage();

    // Otros métodos
    bool write(const char *outputFileName, std::vector<std::pair<int, int>> selectedLines, float radIncrement, int rBins);
    int getXDim();
    int getYDim();
    int getNumColors();
    unsigned char* getPixels();

    // Variables miembro
    int x_dim;
    int y_dim;
    int num_colors;
    unsigned char *pixels;
    std::vector<unsigned char> color;
};

#endif // PGM_H

Overwriting pgm.h


In [28]:
%%writefile houghBase.cu
/*
 ============================================================================
 Author        : G. Barlas
 Version       : 1.0
 Last modified : December 2014
 License       : Released under the GNU GPL 3.0
 Description   :
 To build use  : make
 ============================================================================
 */
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda.h>
#include <string.h>
#include "pgm.h"

const int degreeInc = 2;
const int degreeBins = 180 / degreeInc;
const int rBins = 100;
const float radInc = degreeInc * M_PI / 180;
//*****************************************************************
// The CPU function returns a pointer to the accummulator
void CPU_HoughTran (unsigned char *pic, int w, int h, int **acc)
{
  float rMax = sqrt (1.0 * w * w + 1.0 * h * h) / 2;  //(w^2 + h^2)/2, radio max equivalente a centro -> esquina
  *acc = new int[rBins * degreeBins];            //el acumulador, conteo depixeles encontrados, 90*180/degInc = 9000
  memset (*acc, 0, sizeof (int) * rBins * degreeBins); //init en ceros
  int xCent = w / 2;
  int yCent = h / 2;
  float rScale = 2 * rMax / rBins;

  for (int i = 0; i < w; i++) //por cada pixel
    for (int j = 0; j < h; j++) //...
      {
        int idx = j * w + i;
        if (pic[idx] > 0) //si pasa thresh, entonces lo marca
          {
            int xCoord = i - xCent;
            int yCoord = yCent - j;  // y-coord has to be reversed
            float theta = 0;         // actual angle
            for (int tIdx = 0; tIdx < degreeBins; tIdx++) //add 1 to all lines in that pixel
              {
                float r = xCoord * cos (theta) + yCoord * sin (theta);
                int rIdx = (r + rMax) / rScale;
                (*acc)[rIdx * degreeBins + tIdx]++; //+1 para este radio r y este theta
                theta += radInc;
              }
          }
      }
}

//*****************************************************************
// TODO usar memoria constante para la tabla de senos y cosenos
// inicializarlo en main y pasarlo al device
//__constant__ float d_Cos[degreeBins];
//__constant__ float d_Sin[degreeBins];

//*****************************************************************
//TODO Kernel memoria compartida
// __global__ void GPU_HoughTranShared(...)
// {
//   //TODO
// }
//TODO Kernel memoria Constante
// __global__ void GPU_HoughTranConst(...)
// {
//   //TODO
// }

// GPU kernel. One thread per image pixel is spawned.
// The accummulator memory needs to be allocated by the host in global memory
__global__ void GPU_HoughTran (unsigned char *pic, int w, int h, int *acc, float rMax, float rScale, float *d_Cos, float *d_Sin)
{
  int gloID = blockIdx.x * blockDim.x + threadIdx.x;
  if (gloID >= w * h) return; // omitir hilos extra

  int xCent = w / 2;
  int yCent = h / 2;
  int xCoord = gloID % w - xCent;
  int yCoord = yCent - gloID / w;

  if (pic[gloID] > 0)
    {
      for (int tIdx = 0; tIdx < degreeBins; tIdx++)
        {
          float r = xCoord * d_Cos[tIdx] + yCoord * d_Sin[tIdx];
          int rIdx = (r + rMax) / rScale;
          atomicAdd(acc + (rIdx * degreeBins + tIdx), 1);
        }
    }
}


//*****************************************************************
int main (int argc, char **argv)
{
  int i;

  PGMImage inImg (argv[1]);

  int *cpuht;
  int w = inImg.x_dim;
  int h = inImg.y_dim;

  float* d_Cos;
  float* d_Sin;

  cudaMalloc ((void **) &d_Cos, sizeof (float) * degreeBins);
  cudaMalloc ((void **) &d_Sin, sizeof (float) * degreeBins);

  // CPU calculation
  CPU_HoughTran(inImg.pixels, w, h, &cpuht);

  // pre-compute values to be stored
  float *pcCos = (float *) malloc (sizeof (float) * degreeBins);
  float *pcSin = (float *) malloc (sizeof (float) * degreeBins);
  float rad = 0;
  for (i = 0; i < degreeBins; i++)
  {
    pcCos[i] = cos (rad);
    pcSin[i] = sin (rad);
    rad += radInc;
  }

  float rMax = sqrt (1.0 * w * w + 1.0 * h * h) / 2;
  float rScale = 2 * rMax / rBins;

  // TODO eventualmente volver memoria global
  cudaMemcpy(d_Cos, pcCos, sizeof (float) * degreeBins, cudaMemcpyHostToDevice);
  cudaMemcpy(d_Sin, pcSin, sizeof (float) * degreeBins, cudaMemcpyHostToDevice);

  // setup and copy data from host to device
  unsigned char *d_in, *h_in;
  int *d_hough, *h_hough;

  h_in = inImg.pixels; // h_in contiene los pixeles de la imagen

  h_hough = (int *) malloc (degreeBins * rBins * sizeof (int));

  cudaMalloc ((void **) &d_in, sizeof (unsigned char) * w * h);
  cudaMalloc ((void **) &d_hough, sizeof (int) * degreeBins * rBins);
  cudaMemcpy (d_in, h_in, sizeof (unsigned char) * w * h, cudaMemcpyHostToDevice);
  cudaMemset (d_hough, 0, sizeof (int) * degreeBins * rBins);

  // execution configuration uses a 1-D grid of 1-D blocks, each made of 256 threads
  //1 thread por pixel
  int blockNum = ceil (w * h / 256);
  GPU_HoughTran <<< blockNum, 256 >>> (d_in, w, h, d_hough, rMax, rScale, d_Cos, d_Sin);

  // get results from device
  cudaMemcpy (h_hough, d_hough, sizeof (int) * degreeBins * rBins, cudaMemcpyDeviceToHost);

  // compare CPU and GPU results
  for (i = 0; i < degreeBins * rBins; i++)
  {
    if (cpuht[i] != h_hough[i])
      printf ("Calculation mismatch at : %i %i %i\n", i, cpuht[i], h_hough[i]);
  }
  printf("Done!\n");

  // Liberación de memoria en el host y dispositivo
  free(cpuht);
  free(pcCos);
  free(pcSin);
  free(h_hough);

  cudaFree(d_Cos);
  cudaFree(d_Sin);
  cudaFree(d_in);
  cudaFree(d_hough);

  return 0;
}

Overwriting houghBase.cu


In [29]:
!nvcc -o houghBase houghBase.cu pgm.cpp -ljpeg
!./houghBase runway.pgm

Calculation mismatch at : 1803 1446 1445
Calculation mismatch at : 1893 1506 1507
Calculation mismatch at : 5931 1653 1654
Calculation mismatch at : 6021 1816 1815
Calculation mismatch at : 6104 1642 1641
Calculation mismatch at : 6194 1586 1587
Done!
